## Using a benchmark problem
This tutorial shows how to use and query a benchmark problem.

In [1]:
import sys
sys.path.append('../build')
import numpy as np
import libry as ry

The following loads a benchmark problem, from which we get the ry.MathematicalProgram interface

In [2]:
benchmark = ry.OptBenchmark_InvKin_Endeff("../rai/test/KOMO/switches/model2.g", True)
nlp = benchmark.get()

In [3]:
help(nlp)

Help on MathematicalProgram in module libry object:

class MathematicalProgram(pybind11_builtins.pybind11_object)
 |  Method resolution order:
 |      MathematicalProgram
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  evaluate(...)
 |      evaluate(self: libry.MathematicalProgram, arg0: rai::Array<T>) -> Tuple[rai::Array<T>, rai::Array<T>]
 |      
 |      query the NLP at a point $x$; returns the tuple $(phi,J)$, which is the feature vector and its Jacobian; features define cost terms, sum-of-square (sos) terms, inequalities, and equalities depending on 'getFeatureTypes'
 |  
 |  getBounds(...)
 |      getBounds(self: libry.MathematicalProgram) -> Tuple[rai::Array<T>, rai::Array<T>]
 |      
 |      returns the tuple $(b_{lo},b_{up})$, where both vectors are of same dimensionality of $x$ (or size zero, if there ar

The first method to use is `getInitializationSample`, which returns an initial x

In [4]:
x = nlp.getInitializationSample()

The most important method is `evaluate(x)`, which returns a tuple `(phi,J)`

In [5]:
(phi,J) = nlp.evaluate(x)
print('phi=', phi, '\nJ=', J)

phi= [  2.51274549e-03  -9.67901608e-03  -9.44928593e-04  -8.69411563e-03
  -2.31881025e-03  -1.25671520e-03   1.52480540e-02  -5.02814852e-03
  -3.25327944e+01  -1.00178475e+01   7.81402580e+01  -8.41324331e+01
  -1.37370498e-03  -4.59469363e+01  -8.09249491e+01] 
J= [[  1.00000000e+00   1.00000000e+00   1.00000000e+00   1.00000000e+00
    1.00000000e+00   1.00000000e+00   1.00000000e+00   1.00000000e+00]
 [ -4.93820208e-17   6.93889390e-15  -7.67411850e-17   3.67721314e+00
    2.80799393e-02   2.20038023e+01  -1.13265232e-02   1.49992561e+01]
 [ -1.72482990e-02  -2.40398266e+01  -8.81851364e-02   4.98314905e+01
   -9.22801022e-02   5.51205499e+01   7.35456382e-02   4.78401070e+01]
 [  1.20210266e-01  -2.74670740e+01   1.78475318e-02   2.74672056e+01
    3.96294814e-18   0.00000000e+00  -4.54761577e+01  -1.15573142e-03]
 [  7.42529628e-04   0.00000000e+00   1.15573142e-03  -7.42529628e-04
    4.54761577e+01  -0.00000000e+00   0.00000000e+00  -4.08175377e-02]
 [ -1.03733722e-06  -5.405

In [6]:
#note the dimensionalities:
print('dim(x)=', nlp.getDimension())
print('dim(x)=', x.shape)
print('dim(phi)=', phi.shape)
print('dim(J)=', J.shape)

dim(x)= 8
dim(x)= (8,)
dim(phi)= (15,)
dim(J)= (15, 8)


The features `phi` define all cost terms and inequality/equality constraint values. The feature types tell you, which feature entries refer to costs/constraints. For each feature, an integer is returned -- but actually it is an enum. So you can test equality with ry.OT.f, ry.OT.sos, ry.OT.ineq, or ry.OT.eq

In [7]:
types = nlp.getFeatureTypes()
print(types)
print(types[0] == ry.OT.f)
print(types[0] == ry.OT.sos)
print(types[0] == ry.OT.ineq)
print(types[0] == ry.OT.eq)

[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
False
True
False
False


So, in our case, all features are sum-of-square features, the total cost function is $\sum_i \phi_i^2$, and we have no constraints.

Finally, the benchmark problem implement a report, which should help you to get more insight in what $x$ means and what is computed. For high verbosity it also displays something - in our case the robot in pose $x$.

In [8]:
# the report method generally returns some info string - internal information given by the benchmark implementation
print(nlp.report(4))

KOMO Problem:
  x-dim:8  dual-dim:0
  T:1 k:1 phases:1 stepsPerPhase:1 tau:1
  #timeSlices:2 #totalDOFs:8 #frames:98
    times:[1]
  computeCollisions:0
    TASK 'F_qItself/1-#16' (-1..0)  type:sos  order:1  target:[]  scale:[1, 1, 1, 1, 1, 1, 1, 1]
    TASK 'F_PositionDiff/0-gripper-box' (0..0)  type:sos  order:0  target:[]  scale:[100]
    TASK 'F_VectorDiff/0-gripper-box' (0..0)  type:sos  order:0  target:[]  scale:[100]
    TASK 'F_ScalarProduct/0-gripper-box' (0..0)  type:sos  order:0  target:[]  scale:[100]
{F_qItself/1-#16:{ order:1, type:sos, sos:0.000441219 },
F_PositionDiff/0-gripper-box:{ order:0, type:sos, sos:7264.64 },
F_VectorDiff/0-gripper-box:{ order:0, type:sos, sos:9189.39 },
F_ScalarProduct/0-gripper-box:{ order:0, type:sos, sos:6548.85 },
sos:23002.9,
ineq:0,
eq:0,
f:0}


## demo for testing
Calling the rai optimizer as below is just for testing - you're supposed to instead implement your own solvers!

In [9]:
#setup a solver with that problem
solver = ry.NLP_Solver()
solver.setProblem(nlp)
solver.setSolver(ry.NLP_SolverID.newton)

In [10]:
#solve it
solver.solve(True)

array([-0.06007811,  0.68417406,  0.21718386, -1.95674602, -0.27367314,
        2.61292275, -0.96180571,  0.04997098])

In [11]:
solver.getTrace_x()

array([[ -6.59519748e-04,  -1.01551361e+00,  -8.99614777e-03,
         -1.99043320e+00,   5.16360263e-03,   2.00583468e+00,
          1.89769477e-02,   4.46248468e-02],
       [  5.85617797e-02,  -1.06575903e+00,  -6.40044452e-03,
         -2.19043320e+00,  -5.21958219e-02,   2.08233655e+00,
         -1.48150662e-01,   4.49337715e-02],
       [  1.13406903e-01,  -1.08503611e+00,  -9.68679952e-03,
         -2.39043320e+00,  -1.04121911e-01,   2.16874951e+00,
         -2.84259165e-01,   4.53134794e-02],
       [  1.70539662e-01,  -1.04346010e+00,  -2.51659151e-02,
         -2.59043320e+00,  -1.59762739e-01,   2.28526026e+00,
         -4.14954699e-01,   4.58067510e-02],
       [  2.29935962e-01,  -8.46741053e-01,  -6.83318079e-02,
         -2.77219312e+00,  -2.29310929e-01,   2.48526026e+00,
         -5.56186543e-01,   4.65180881e-02],
       [  2.46484026e-01,  -6.46741053e-01,  -9.89479162e-02,
         -2.77623264e+00,  -2.56968046e-01,   2.60831483e+00,
         -6.07399043e-01,   4.6

In [12]:
#the first is total cost, 2nd is ineq, 3rd is eq
solver.getTrace_costs()

array([[  2.37623796e+04,   0.00000000e+00,   0.00000000e+00],
       [  1.99145542e+04,   0.00000000e+00,   0.00000000e+00],
       [  1.58411462e+04,   0.00000000e+00,   0.00000000e+00],
       [  1.13791842e+04,   0.00000000e+00,   0.00000000e+00],
       [  6.43505966e+03,   0.00000000e+00,   0.00000000e+00],
       [  4.69352704e+03,   0.00000000e+00,   0.00000000e+00],
       [  3.42413739e+03,   0.00000000e+00,   0.00000000e+00],
       [  2.40067813e+03,   0.00000000e+00,   0.00000000e+00],
       [  1.55464651e+03,   0.00000000e+00,   0.00000000e+00],
       [  8.58967675e+02,   0.00000000e+00,   0.00000000e+00],
       [  3.37507844e+02,   0.00000000e+00,   0.00000000e+00],
       [  6.62816797e+01,   0.00000000e+00,   0.00000000e+00],
       [  5.82001911e+00,   0.00000000e+00,   0.00000000e+00],
       [  4.26928921e+00,   0.00000000e+00,   0.00000000e+00],
       [  4.26788647e+00,   0.00000000e+00,   0.00000000e+00]])

In [13]:
print(nlp.report(4))

KOMO Problem:
  x-dim:8  dual-dim:0
  T:1 k:1 phases:1 stepsPerPhase:1 tau:1
  #timeSlices:2 #totalDOFs:8 #frames:98
    times:[1]
  computeCollisions:0
    TASK 'F_qItself/1-#16' (-1..0)  type:sos  order:1  target:[]  scale:[1, 1, 1, 1, 1, 1, 1, 1]
    TASK 'F_PositionDiff/0-gripper-box' (0..0)  type:sos  order:0  target:[]  scale:[100]
    TASK 'F_VectorDiff/0-gripper-box' (0..0)  type:sos  order:0  target:[]  scale:[100]
    TASK 'F_ScalarProduct/0-gripper-box' (0..0)  type:sos  order:0  target:[]  scale:[100]
{F_qItself/1-#16:{ order:1, type:sos, sos:4.26473 },
F_PositionDiff/0-gripper-box:{ order:0, type:sos, sos:0.00288605 },
F_VectorDiff/0-gripper-box:{ order:0, type:sos, sos:0.00017454 },
F_ScalarProduct/0-gripper-box:{ order:0, type:sos, sos:9.29926e-05 },
sos:4.26789,
ineq:0,
eq:0,
f:0}


In [17]:
nlp = 0
benchmark = 0
solver = 0